In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
import re

In [149]:
def selenium_getsource(link):
    browser = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver")
    browser.get(link)
    source = browser.page_source
    browser.close()
    return source

def retrieve_pergikuliner(url):
    soup = BeautifulSoup(selenium_getsource(url))
    restaurant = pd.Series([])
    #imprecise
    restaurant[0] = soup.h1.get_text()
    restaurant[1] = soup.find(attrs={'class':'info-list'}).li.get_text()
    restaurant[2] = soup(attrs={'class': 'left'})[1].get_text()
    restaurant[3] = soup.link.get('href')

    #not cleared
    restaurant[4] = soup.find(class_='info-list').find_all('li')[-1].get_text()
    #check multi-valued attributes
    restaurant[5] = re.search('(?<=content=).+?"', str(soup.find(itemprop='paymentAccepted'))).group()
    #precise
    promo = soup.find(attrs={'class': 'promotional-text'})
    restaurant[6] = str(promo.get_text) if promo is not None else None
    restaurant[7] = soup.find(string=re.compile('Rp. .* '))
    schedule = soup.find(attrs={'itemprop': 'openingHours'})
    restaurant[8] = str(schedule.get_text) if schedule is not None else None

    #cleared
    restaurant[9] = ','.join([result.get_text() for result in soup(class_='checked')])
    restaurant[10] = ','.join([result.get_text() for result in soup(class_='unchecked')])
    restaurant[11] = soup.find(href=re.compile("tel.*")).get_text()
    restaurant[12] = soup.find(itemprop='ratingValue').get_text()
    #kalo dibawah 4 ratingnya suka ga muncul
    restaurant[13] = ','.join([i.getText().strip() for i in soup.find_all(class_='rate-box-bottom best-rating')])
    return restaurant


In [ ]:
restaurants_compiled = []

for pageindex in range(1, 126):
    soup = BeautifulSoup(selenium_getsource(f'https://pergikuliner.com/restaurants?page={pageindex}'))
    links = soup.find_all(href=re.compile('restaurants'))
    for i in range(1, len(links), 2):
        urltest = re.search('/.*/.*/.*(?=")', str(links[i]))
        if urltest is None:
            pass
        else:
            urlext = urltest.group()
            restaurants_compiled.append(retrieve_pergikuliner(f'https://pergikuliner.com{urlext}'))


restaurants_dataset = pd.concat(restaurants_compiled, axis=1).T

In [10]:
def driver_setup():
    
    # options = webdriver.ChromeOptions()
    # options.add_argument('--headless')
    # options.add_argument('--no-sandbox')
    # options.add_argument('--disable-dev-shm-usage')
    
    driver = webdriver.Chrome()

    return driver

def check_review_state(driver):
    WebDriverWait(driver, timeout=10).until("js has done loading")
    show_next_review = driver.find_element(By.LINK_TEXT, "Tampilkan review sebelumnya")
    while show_next_review:
        show_next_review.click()
        check_review_state(driver)

def review_fetcher(url, driver):
    #build a check function to see if there is a "Tampilkan review sebelumnya" button. 
    #if it exists:
    #   open it. 
    #   initialize wait until all the comments have been loaded.
    #   then reinitialize the check function.
    #if it doesn't, then pass the operations.

    #or.. just initialize a webdriverwait operation.
    driver.get(url)
    try:
        
        ins = driver.find_element(By.LINK_TEXT, "Tampilkan review sebelumnya").click()
        WebDriverWait(driver, timeout=10).until(document_initialised)
    #two types of exceptions: timeout error and object not found error.
    #timeout error should be raised
    #object not found should be passed
    except:
        pass
    #then loop through all of them

driver = driver_setup()

review_fetcher('https://pergikuliner.com/restaurants/jakarta/shaburi-dan-kintan-buffet-scbd', driver)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"Tampilkan review sebelumnya"}
  (Session info: chrome=101.0.4951.64)
Stacktrace:
#0 0x55c2edfe9533 <unknown>
#1 0x55c2edd481e8 <unknown>
#2 0x55c2edd7e2d7 <unknown>
#3 0x55c2edd7e4a1 <unknown>
#4 0x55c2eddb1494 <unknown>
#5 0x55c2edd9bf6d <unknown>
#6 0x55c2eddaf224 <unknown>
#7 0x55c2edd9be33 <unknown>
#8 0x55c2edd7171a <unknown>
#9 0x55c2edd72875 <unknown>
#10 0x55c2ee02de1d <unknown>
#11 0x55c2ee031751 <unknown>
#12 0x55c2ee01807e <unknown>
#13 0x55c2ee032388 <unknown>
#14 0x55c2ee00cfe0 <unknown>
#15 0x55c2ee04e748 <unknown>
#16 0x55c2ee04e8c8 <unknown>
#17 0x55c2ee06870d <unknown>
#18 0x7fe1a5cedb43 <unknown>
